Rahmanda Afebrio Yuris Soesatyo - 1103223024 / Midterm-Deep Learning-Regression

In [1]:
import gdown

gdown.download("https://drive.google.com/uc?id=1f8eaAZY-7YgFxLcrL3OkvSRa3onNNLb9")

Downloading...
From (original): https://drive.google.com/uc?id=1f8eaAZY-7YgFxLcrL3OkvSRa3onNNLb9
From (redirected): https://drive.google.com/uc?id=1f8eaAZY-7YgFxLcrL3OkvSRa3onNNLb9&confirm=t&uuid=eba977a2-eafa-4455-ac63-69c2316cd777
To: c:\College\Deep Learning\Midterm\midterm-regresi-dataset.csv
100%|██████████| 443M/443M [00:58<00:00, 7.61MB/s] 


'midterm-regresi-dataset.csv'

In [ ]:
import pandas as pd


file_path = 'midterm-regresi-dataset.csv'
df = pd.read_csv(file_path)

df.head()

,2001,49.94357,21.47114,73.0775,8.74861,-17.40628,-13.09905,-25.01202,-12.23257,7.83089,...,13.0162,-54.40548,58.99367,15.37344,1.11144,-23.08793,68.40795,-1.82223,-27.46348,2.26327
0,2001,48.73215,18.42930,70.32679,12.94636,-10.32437,-24.83777,8.76630,-0.92019,18.76548,...,5.66812,-19.68073,33.04964,42.87836,-9.90378,-32.22788,70.49388,12.04941,58.43453,26.92061
1,2001,50.95714,31.85602,55.81851,13.41693,-6.57898,-18.54940,-3.27872,-2.35035,16.07017,...,3.03800,26.05866,-50.92779,10.93792,-0.07568,43.20130,-115.00698,-0.05859,39.67068,-0.66345
2,2001,48.24750,-1.89837,36.29772,2.58776,0.97170,-26.21683,5.05097,-10.34124,3.55005,...,34.57337,-171.70734,-16.96705,-46.67617,-12.51516,82.58061,-72.08993,9.90558,199.62971,18.85382
3,2001,50.97020,42.20998,67.09964,8.46791,-15.85279,-16.81409,-12.48207,-9.37636,12.63699,...,9.92661,-55.95724,64.92712,-17.72522,-1.49237,-7.50035,51.76631,7.88713,55.66926,28.74903
4,2001,50.54767,0.31568,92.35066,22.38696,-25.51870,-19.04928,20.67345,-5.19943,3.63566,...,6.59753,-50.69577,26.02574,18.94430,-0.33730,6.09352,35.18381,5.00283,-11.02257,0.02263


Terlihat bahwasannya dataset tidak memiliki header, lanjut ke preprocessing

In [ ]:
print("df.shape:", df.shape) 

df.shape: (515344, 91)


In [4]:
import pandas as pd
import numpy as np
import time
import os

# TensorFlow / Keras Imports
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # Suppress TensorFlow warnings
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Scikit-learn Imports for Preprocessing and Evaluation
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [5]:
# Since the dataset has no header, we manually create column names.
feature_cols = [f'feature_{i}' for i in range(1, 91)]
column_names = ['target'] + feature_cols

# Load the dataset
try:
    df = pd.read_csv(file_path, header=None, names=column_names)
except FileNotFoundError:
    print(f"Error: File not found at {file_path}. Please ensure the file is in the correct directory.")
    exit()

print(f"Dataset loaded. Total shape: {df.shape}")

Dataset loaded. Total shape: (515345, 91)


Testing split data train beserta standarisasi target

In [6]:
# Use a sample for demonstration. Change SAMPLE_FRACTION to 1.0 to use the full dataset.
SAMPLE_FRACTION = 1

if SAMPLE_FRACTION < 1.0:
    print(f"Status: Using a sample. Sampling {SAMPLE_FRACTION*100:.0f}% of the data.")
    df_sample = df.sample(frac=SAMPLE_FRACTION, random_state=42).copy()
else:
    print("Status: Using the full dataset (SAMPLE_FRACTION = 1.0).")
    df_sample = df.copy()

print(f"Sampled Dataset Shape: {df_sample.shape}")

# Separate features (X) and target (y)
X = df_sample.drop('target', axis=1)
y = df_sample['target'].copy()
numerical_features = X.columns.tolist()

# The target variable (year, e.g., 2001) is often large.
# Standardizing the target helps Deep Learning models converge faster.
target_mean = y.mean()
target_std = y.std()

# Target Standardization: y_scaled = (y - mean) / std
y_scaled = (y - target_mean) / target_std
print(f"Target variable (Year) standardized. Mean: {target_mean:.2f}, Std Dev: {target_std:.2f}")

# Split data into training and testing sets
X_train, X_test, y_train_scaled, y_test_scaled = train_test_split(
    X, y_scaled, test_size=0.2, random_state=42
)
y_test_original = y.loc[y_test_scaled.index] # Keep original years for final reporting

print(f"Training set size: {X_train.shape[0]} rows")
print(f"Test set size: {X_test.shape[0]} rows")

Status: Using the full dataset (SAMPLE_FRACTION = 1.0).
Sampled Dataset Shape: (515345, 91)
Target variable (Year) standardized. Mean: 1998.40, Std Dev: 10.93
Training set size: 412276 rows
Test set size: 103069 rows


In [8]:
# We only need scaling for the features, as the target is handled above.
numerical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Create a ColumnTransformer to apply the scaling to all 90 features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_pipeline, numerical_features)
    ],
    remainder='passthrough',
    n_jobs=-1
)

# Apply preprocessing and convert to NumPy array for TensorFlow
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

print(f"Features processed. Training shape: {X_train_processed.shape}")

Features processed. Training shape: (412276, 90)


In [9]:
# --- 4. TensorFlow Model Definition (MLP) ---
print("\n--- 4. TensorFlow Model Definition (MLP) ---")

input_shape = X_train_processed.shape[1]
print(f"Input Features: {input_shape}")

# Simple Multi-Layer Perceptron (MLP) for Regression
def create_mlp_model(input_dim):
    model = Sequential([
        # Input Layer: 90 features
        Dense(128, activation='relu', input_shape=(input_dim,)),
        Dropout(0.2), # Dropout for regularization
        
        # Hidden Layer 1
        Dense(64, activation='relu'),
        
        # Hidden Layer 2
        Dense(32, activation='relu'),
        
        # Output Layer: 1 unit for the scaled target value
        Dense(1, activation='linear') 
    ])
    
    # Compile the model
    # Using 'Adam' optimizer and 'mse' loss for regression on scaled target
    model.compile(optimizer=Adam(learning_rate=0.001), 
                  loss='mse', 
                  metrics=['mae', 'mse'])
    return model

model = create_mlp_model(input_shape)
model.summary()


--- 4. TensorFlow Model Definition (MLP) ---
Input Features: 90


c:\College\Deep Learning\Midterm\.venv\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │        11,648 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 22,017 (86.00 KB)

 Trainable params: 22,017 (86.00 KB)

 Non-trainable params: 0 (0.00 B)

Training model dengan early stopping

In [10]:

early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)

EPOCHS = 100 
BATCH_SIZE = 64 

start_time = time.time()
history = model.fit(
    X_train_processed, y_train_scaled,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_split=0.1, # Use 10% of training data for validation
    callbacks=[early_stopping],
    verbose=2 # Verbose 2 shows progress per epoch
)
training_duration = time.time() - start_time

print(f"\nTraining Duration: {training_duration:.2f} seconds")

Epoch 1/100
5798/5798 - 7s - 1ms/step - loss: 0.6994 - mae: 0.5850 - mse: 0.6994 - val_loss: 0.6473 - val_mae: 0.5495 - val_mse: 0.6473
Epoch 2/100
5798/5798 - 6s - 1ms/step - loss: 0.6523 - mae: 0.5594 - mse: 0.6523 - val_loss: 0.6295 - val_mae: 0.5453 - val_mse: 0.6295
Epoch 3/100
5798/5798 - 6s - 1ms/step - loss: 0.6420 - mae: 0.5549 - mse: 0.6420 - val_loss: 0.6271 - val_mae: 0.5435 - val_mse: 0.6271
Epoch 4/100
5798/5798 - 6s - 1ms/step - loss: 0.6335 - mae: 0.5505 - mse: 0.6335 - val_loss: 0.6238 - val_mae: 0.5380 - val_mse: 0.6238
Epoch 5/100
5798/5798 - 6s - 1ms/step - loss: 0.6274 - mae: 0.5473 - mse: 0.6274 - val_loss: 0.6188 - val_mae: 0.5425 - val_mse: 0.6188
Epoch 6/100
5798/5798 - 7s - 1ms/step - loss: 0.6230 - mae: 0.5457 - mse: 0.6230 - val_loss: 0.6196 - val_mae: 0.5474 - val_mse: 0.6196
Epoch 7/100
5798/5798 - 7s - 1ms/step - loss: 0.6185 - mae: 0.5436 - mse: 0.6185 - val_loss: 0.6151 - val_mae: 0.5353 - val_mse: 0.6151
Epoch 8/100
5798/5798 - 6s - 1ms/step - loss: 0.

Evaluasi pada test set

In [11]:
# Make predictions on the scaled test set
y_pred_scaled = model.predict(X_test_processed, verbose=0).flatten()

y_pred_denorm = (y_pred_scaled * target_std) + target_mean

# Calculate evaluation metrics using the DENORMALIZED predictions and original labels
mse = mean_squared_error(y_test_original, y_pred_denorm)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test_original, y_pred_denorm)
r2 = r2_score(y_test_original, y_pred_denorm)

print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"R-squared (R²): {r2:.4f}")

Mean Squared Error (MSE): 73.0717
Root Mean Squared Error (RMSE): 8.5482
Mean Absolute Error (MAE): 5.8431
R-squared (R²): 0.3860
